In [2]:
import numpy as np
import pandas as pd 
complete_data = pd.read_parquet('pre_processed_news.parquet')



In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

# split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(complete_data['cleaned_content'], complete_data['type'], test_size=0.2, random_state=42)
y_train = y_train.apply(lambda x: True if x in ["reliable", "political"] else False)
y_test = y_test.apply(lambda x: True if x in ["reliable", "political"] else False)

# Let's split test into test and validation:
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

print('X_train: ', X_train.shape)
print('X_test: ', X_test.shape)
print('y_train: ', y_train.shape)
print('y_test: ', y_test.shape)
print('X_val: ', X_val.shape)
print('y_val: ', y_val.shape)

X_train:  (588414,)
X_test:  (73552,)
y_train:  (588414,)
y_test:  (73552,)
X_val:  (73552,)
y_val:  (73552,)


In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [6]:
import pickle

with open('tokenizer.pickle', 'rb') as handle:
    loaded_tokenizer = pickle.load(handle)

with open('X_train_seq.pickle', 'rb') as handle:
    X_train_seq = pickle.load(handle)

# Loading X_test_seq
with open('X_test_seq.pickle', 'rb') as handle:
    X_test_seq = pickle.load(handle)

In [7]:
print(max([len(x) for x in X_train_seq]))
print(np.mean([len(x) for x in X_train_seq]))
print(np.percentile([len(x) for x in X_train_seq], 80))

31946
504.7287403086942
756.0


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense

In [10]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

gpus = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpus))

TensorFlow version: 2.16.1
Num GPUs Available:  0


In [11]:
X_train_pad = pad_sequences(X_train_seq, maxlen=512, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=512, padding='post')

In [12]:
total_samples = len(X_train_pad)  # Total number of samples in your training data
batch_size = 512  # Assuming this is the batch size you've chosen
steps_per_epoch = total_samples // batch_size
steps_per_epoch, total_samples

(1149, 588414)

In [13]:
with tf.device('/GPU:0'):
    model = Sequential([
        Embedding(input_dim=25000, output_dim=64, input_shape=(800,)),  # Adjust these parameters as needed
        GRU(units=256, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), 
        GRU(units=32, dropout=0.2, recurrent_dropout=0.2),  # Adjusted units, added dropout
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'], steps_per_execution=32)

model.summary()

c:\.KU\Grundlæggende Data Science\Eksamen\.venv\lib\site-packages\keras\src\layers\core\embedding.py:81: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 800, 64)        │     1,600,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 800, 256)       │       247,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 32)             │        27,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │         4,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,879,489 (7.17 MB)

 Trainable params: 1,879,489 (7.17 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
from tensorflow.keras.utils import Sequence
import numpy as np   

class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y

train_gen = DataGenerator(X_train_pad, y_train, batch_size)
test_gen = DataGenerator(X_test_pad, y_test, batch_size)

In [ ]:
history = model.fit(X_train_pad, y_train, epochs=1,batch_size=batch_size, validation_split=0.2)
